# Encoder-Decoder LSTMによる非線形歪補償
Encoder-Decoder LSTMによる補償

In [15]:
#import
import sys
import time
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

sys.path.append('../')
from pyopt.util import save_pickle, load_pickle

# 1. Preprocessing

## 1.1 データの整形

In [3]:
def data_shaping(input_signal, signal, max_tap, tap):
    """
    input_signal: 伝送前の信号
    signal: 伝送後の信号
    max_tap: 最大の同時入力シンボル数
    tap: 同時入力シンボル数
    
    signal = [x_0, x_1, ... , x_(n-1)]
      |
      |
      v
    x = [[x_0, x_1, ... , x_tap-1],
            [x_1, x_2, ..., x_tap],
                   .
                   .
                   .
            [x_(n-tap), x_(n-tap+1), ..., x(n-1)]]
      |
      |
      v
    x = [[i_0, q_0, i_1, q_1, ... , i_(tap-1), q_(tap-1)],
            [i_1, q_1, i_2, q_2, ... , i_tap, q_tap],
                   .
                   .
                   .
            [i_(n-tap), q_(n-tap), i_(n-tap+1), q_(n-tap+1), ..., i_(n-1), q_(n-1)]] (batch, input_dim) input_dim = tap * 2
    
    y  (batch, output_dim) output_dim = 2
    """
    
    x = np.zeros((len(input_signal) - (max_tap - 1), tap * 2), dtype=float)
    y = np.zeros((len(input_signal) - (max_tap - 1), 2), dtype=float)
    for i, j in enumerate(np.arange(int((max_tap - 1) / 2), len(input_signal) - int((max_tap - 1) / 2))):
        x[i, 0::2] = signal[j - int((tap - 1) / 2): j + int((tap - 1) / 2) + 1].real
        x[i, 1::2] = signal[j - int((tap - 1) / 2): j + int((tap - 1) / 2) + 1].imag
        y[i, 0] = input_signal[j].real
        y[i, 1] = input_signal[j].imag
    return x, y

In [9]:
def data_shaping2(input_signal, signal, max_tap, tap, lstm_tap):
    x, y = data_shaping(input_signal, signal, max_tap, tap)
    x = x[:x.shape[0] // lstm_tap * lstm_tap].reshape([x.shape[0] // lstm_tap, lstm_tap, -1])
    y = y[:y.shape[0] // lstm_tap * lstm_tap].reshape([y.shape[0] // lstm_tap, lstm_tap, -1])
    return x, y

In [12]:
#動作確認
tap = 5
max_tap = 51
lstm_tap = 100

df_dir = '../data/input/prbs.csv'
df = pd.read_csv(df_dir, index_col=0)  # dataframe読み込み
condition = (df['N']==13) & (df['itr']==1) & (df['form']=='RZ16QAM') & (df['n']==32) & (df['equalize']==False) & (df['baudrate']==28) & (df['PdBm']==1)
sgnl = load_pickle(df[condition].iloc[0]['data_path'])  # dataframeから条件と合う行を取得し,pickleの保存先(data_path)にアクセス
lc = sgnl.linear_compensation(500, sgnl.signal['x_500'])
x, y = data_shaping2(sgnl.signal['x_0'][16::32], lc[16::32], max_tap, tap, lstm_tap)  # ANNに入力できるようにデータを整形

print('x size: ', x.shape)
print('y size: ', y.shape)

x size:  (19, 100, 10)
y size:  (19, 100, 2)


## 1.2 平均,標準偏差の計算

In [13]:
mean = np.mean(x)
std = np.std(x)

print('mean: ', mean)
print('std: ', std)

mean:  1175.6117983961424
std:  52142.10689249052


# 2. Dataset定義

In [16]:
class Dataset(data.Dataset):
    def __init__(self, x, y, mean, std):
        self.x, self.y, self.mean, self.std = x, y, mean, std
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        x = (x - self.mean) / self.std
        y = (y - self.mean) / self.std
        return torch.Tensor(x), torch.Tensor(y)

In [17]:
#動作確認
train_dataset = Dataset(x=x, y=y, mean=mean, std=std)

index = 0
x_normalized, y_normalized = train_dataset.__getitem__(index)
x_array = x_normalized.detach().numpy()

print('mean: ', np.mean(x_array))
print('std: ', np.std(x_array))
print(x_normalized)
print(y_normalized)

mean:  0.0076609305
std:  0.9650807
tensor([[-1.3992,  0.1550, -0.9966,  0.9840, -0.9993,  0.9950,  0.5773, -1.8108,
         -0.3445,  0.6298],
        [-0.9966,  0.9840, -0.9993,  0.9950,  0.5773, -1.8108, -0.3445,  0.6298,
         -0.9276,  0.9479],
        [-0.9993,  0.9950,  0.5773, -1.8108, -0.3445,  0.6298, -0.9276,  0.9479,
         -0.2741, -1.4384],
        [ 0.5773, -1.8108, -0.3445,  0.6298, -0.9276,  0.9479, -0.2741, -1.4384,
         -0.1860,  0.6089],
        [-0.3445,  0.6298, -0.9276,  0.9479, -0.2741, -1.4384, -0.1860,  0.6089,
         -1.0067,  0.9526],
        [-0.9276,  0.9479, -0.2741, -1.4384, -0.1860,  0.6089, -1.0067,  0.9526,
         -1.8437, -0.6557],
        [-0.2741, -1.4384, -0.1860,  0.6089, -1.0067,  0.9526, -1.8437, -0.6557,
         -1.0636,  0.9630],
        [-0.1860,  0.6089, -1.0067,  0.9526, -1.8437, -0.6557, -1.0636,  0.9630,
         -1.3443,  0.2292],
        [-1.0067,  0.9526, -1.8437, -0.6557, -1.0636,  0.9630, -1.3443,  0.2292,
          1

In [18]:
batch_size = 100

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader}

# 3. Model定義